# Data supplementation

## Imports

In [34]:
import os
os.chdir("g:\\My Drive\\ITC, 5th semester (Thesis)\\Code\\Github_code\\toxicity_detection")

from wiki_scraper import scrape_wiki_text
import nltk
import utils
from tqdm import tqdm
import dacy

## Scrape Wikipedia data

In [2]:
content = scrape_wiki_text("https://da.wikipedia.org/wiki/Transk%C3%B8nnethed")
print("_"*100)
for text in content:
    print(text)

Successfully scraped the webpage with the title: "Transkønnethed"
____________________________________________________________________________________________________
Transkønnethed er en betegnelse for personer, der har en kønsidentitet eller et kønsudtryk, der adskiller sig fra deres fødselskøn.[1][2][3] Nogle transkønnede, som ønsker medicinsk hjælp til at overgå fra et køn til et andet, identificerer sig som transseksuelle.[4][5] Transkønnet, ofte forkortet til blot trans, er også et paraplybegreb: Udover at omfatte personer, hvis kønsidentitet er det modsatte af deres fødselskøn (dvs. transmænd og transkvinder), kan det også anvendes om personer, hvis kønsudtryk ikke er eksklusivt maskulint eller feminint (personer, som er ikke-binære eller genderqueer, heriblandt bikønnede, pankønnede, genderfluid og akønnede).[2][6][7] Blandt andre definitioner af transkønnet er også at inkludere personer, der tilhører et tredje køn, eller konceptualisere transkønnede som et tredje køn.[8][9] Be

1) Search for pages to add (manually selected)
2) Scrape these pages using requests and beautifulsoup4
3) Concatenate to one big text bank
4) Search for word forms in this text bank. Extract the needed number of texts in the correct length.
5) Train model on the new dataset and do bias analysis

Afterwards, try to do both types of mitigation on the oversampled dataset

OR 

Try to rerun the original model on non-oversampled dataset
ASK MANEX!

## Test adding random articles from wikipedia of desired length

In [25]:
url = "https://da.wikipedia.org/wiki/Special:Random"

In [39]:
num_random_to_add = {"0-59": 7+7+7, 
                    "60-179": 6+4+32+3+4+5,
                    "180-419": 4+4+8+5,
                    "420-899": 4+2, 
                    "900-3519": 3+4}

print("Add", sum(num_random_to_add.values()), "samples!")

Add 109 samples!


In [30]:
rd_passages = []

for _ in range(50): # 50 random pages
    content = scrape_wiki_text(url)
    for passage in content:
        rd_passages.append(passage)

Successfully scraped the webpage with the title: "Sindh"
Successfully scraped the webpage with the title: "Ukaleq Slettemark"
Successfully scraped the webpage with the title: "Dansk Vestindien"
Successfully scraped the webpage with the title: "Den mystiske enke"
Successfully scraped the webpage with the title: "Yuto Misao"
Successfully scraped the webpage with the title: "Bindjuzjnik i Korol"
Successfully scraped the webpage with the title: "Alanna"
Successfully scraped the webpage with the title: "Vitalij Melnikov"
Successfully scraped the webpage with the title: "Wimbledon-mesterskaberne 1893"
Successfully scraped the webpage with the title: "Busplade"
Successfully scraped the webpage with the title: "Pigernes nye valg"
Successfully scraped the webpage with the title: "Paulo Costanzo"
Successfully scraped the webpage with the title: "Mike Myers"
Successfully scraped the webpage with the title: "Hymenektomi"
Successfully scraped the webpage with the title: "Santiago de Chile"
Successf

In [15]:
# load daCy model (medium works fine)
nlp = dacy.load("da_dacy_medium_trf-0.2.0") # takes around 4 minutes the first time

In [36]:
# preprocess

# split passages into sentences and preprocess

stop_words = nltk.corpus.stopwords.words('danish')
passage_bank = []
for passage in tqdm(rd_passages):
    sentences = []
    if passage.strip() != "":
        doc = nlp(passage)
        for sent in doc.sents:
            clean_sent = utils.preprocess(str(sent), stop_words)
            if len(clean_sent) > 0: # don't add empty strings
                sentences.append(clean_sent)
        passage_bank.append(sentences)

100%|██████████| 378/378 [05:15<00:00,  1.20it/s]


In [40]:
d = {}
new_examples = []

for length in num_random_to_add: # for each lemma and length we need to deal with

    # range (length bucket)
    length_range = length.split("-")
    length_range = [int(l) for l in length_range]

    # number of nontoxic examples to add
    num_to_add = num_random_to_add[length] # number new nontoxic to add

    # initialize variables
    num_added = 0

    for passage in passage_bank: # for each passage where the lemma appears
        
        if num_added < num_to_add: # only continue if we still need to add more sentences          
            sentence_lengths = [len(sent)+1 for sent in passage] # +1 = space between sentences
            
            # if the full passage is within range, add that
            if length_range[0] <= len(' '.join(passage)) <= length_range[1]:
                #print(type(passage), passage)
                new_examples.append(' '.join(passage))
                num_added += 1

            else:
                for sentence in passage:
                    if length_range[0] <= len(sentence) <= length_range[1]:
                        new_examples.append(sentence)
                        num_added += 1
    
    if num_added < num_to_add:
        print(length_range, num_to_add)
        print(num_added, num_to_add)
        print()

In [41]:
new_examples

['sindh pakistans fire provinser',
 'hovedstaden hyderabad',
 'hovedsprogene sindhi urdu',
 'største by sindh karachi',
 'navnet stammer samme rod indus provinsens vigtigste flod',
 'koordinater °′n °′ø\ufeff \ufeff°n °ø\ufeff',
 'vandt norges cuppen gennemgående sæsonkonkurrence årige',
 'udtaget vinterol beijing kun stillede km samt km',
 'datter uiloq slettemark øystein slettemark',
 'familien slettemark bor geilo norge går topidrætsgymnasium',
 'slutetapen trekanten fragt sukker rom hjemlandet',
 'økonomien dansk vestindien fuldstændig baseret slaveri',
 'jordskælv tropiske cykloner tsunamier forekommer',
 'regntiden varer maj november',
 'højeste bjerg crown mountain skt thomas meter',
 'skt croix areal km² kuperet terræn',
 'højeste punkt øen mount eagle m',
 'sankt croix',
 'slaveoprør sankt croix førte slaveriets afskaffelse',
 'skt jan mindste tre hovedøer dansk vestindien',
 'øen areal km²',
 'hovedstaden hyderabad hovedsprogene sindhi urdu største by sindh karachi',
 'ruiner

In [43]:
len(new_examples)

112